## Inclusione delle librerie

In [ ]:
import math
import pandas as pd

## Definizione del dataset su cui lavorare

In [ ]:
dataset = pd.read_csv("../dataset_base.csv")
# ricorda di mettere un check df.isnull().values.any() e uno script che toglie null se ci sono

## Se presente, eliminazione di 'isFraud'

In [ ]:
dataset = dataset.drop(columns=['isFlaggedFraud'])

## Definizione della nuova classe di 'isFraud' 
Cito: 'The data has several transactions with zero balances in the destination account both before and after a non-zero amount is transacted. The fraction of 
such transactions, where zero likely denotes a missing value, is much larger in fraudulent (50%) compared to genuine transactions (0.06%).'

Per questo motivo, rendo la colonna "isFraud" del mio dataset non più un binario bensì un enumeratore con 3 valori: "0, 1, 2" dove 'Genuino' = 0, 'Ambiguo' = 1, 'Frode' = 2
così da rendere non binario il training set.

Definisco la classe 'Ambiguo' = 1, e ridefinisco 'Frode' = 1


In [ ]:

dataset.loc[dataset['isFraud'] == 1, 'isFraud'] = 2
dataset.loc[(dataset['newbalanceDest'] == 0) & (dataset['oldbalanceDest'] == 0) & (dataset['amount'] != 0)& (dataset['isFraud'] != 2), 'isFraud'] = 1

## Riduzione delle dimensioni del dataset pre-undersampling

In [ ]:
print(dataset.loc[3, 'amount'])
#separo e poi farò l'unione
dataset0 = dataset[dataset['isFraud'] == 0]
dataset1 = dataset[dataset['isFraud'] == 2]
dataset2 = dataset[dataset['isFraud'] == 1]
#mantengo la stessa proporzione di valori che siano frode e non frode
#riduco le dimensioni della classe 1 per renderle la metà. Applicherò un ulteriore undersampling in seguito sulle classi 2 e 0 quindi non serve ridurre anche loro per il momento
n_f = math.floor(len(dataset1)/2)
#randomizzo il dataset così da ottenere, in probabilità, proporzioni simili a quelle iniziali
dataset1 = dataset1.sample(frac=1).reset_index(drop=True)
dataset1 = dataset1[:n_f]
print(dataset1)
print(len(dataset2))
print(len(dataset0))

## Undersampling per ridurre il numero di elementi nella classe 0 e 2

In media il numero di frodi è dell'1%, ma nel dataset abbiamo un numero di frodi molto minore all'1%.
Anche l'1% non è abbastanza, quindi dobbiamo ridurre il numero di elementi nella classe 0 e nella classe 2 in modo tale da rendere len(0) + len(2) = len(1) * 5, mantenendo le proporzioni.
Per fare ciò quindi vado a dividere il numero di elementi nella classe 0 per quelli della classe 2 e li riduco lasciando la loro proporzione attuale.

In [ ]:
n_g = len(dataset0)
n_a = len(dataset2)

rap_g = n_g/(n_g + n_a)
rap_a = n_a/(n_g + n_a)

n_g = math.floor(n_f * 5*rap_g)
n_a = math.floor(n_f * 5*rap_a)

dataset0 = dataset0.sample(frac=1).reset_index(drop=True)
dataset2 = dataset2.sample(frac=1).reset_index(drop=True)

dataset0 = dataset0[:n_g]
dataset2 = dataset2[:n_a]

#print("%s \n %s \n %s", n_f, n_g, n_a)

# Dopo aver completato tutte le operazioni, effettuo il merge del dataset
frames = [dataset0, dataset1, dataset2]
merged = pd.concat(frames)

# Per l'ultima volta randomizzo il dataset così che sia pronto per esser salvato
merged = merged.sample(frac=1).reset_index(drop=True)

## Rename delle colonne e Salvataggio
Rinomino le colonne così che rispettino la convenzione per la denominazione delle colonne di un dataset di tipo 'MyColumn'

In [ ]:
merged.rename(columns={'step' : 'Step','type' : 'Type', 'amount' : 'Amount','nameOrig' : 'NameOrig','oldbalanceOrg' : 'OldBalanceOrig','newbalanceOrig' : 'NewBalanceOrig',
                        'nameDest' : 'NameDest','oldbalanceDest': 'OldBalanceDest','newbalanceDest' : 'NewBalanceDest', 'isFraud' : 'IsFraud'}, inplace=True)
merged.to_csv("../dataset_preprocessato.csv", index=False)

Print vari per controllare se è andato a buon fine

In [ ]:

print(n_a)
print(n_f)
print(n_g)
print(dataset1)
print(dataset2)
print(dataset0)